# TensorFlow Datasets

TFDS provides a collection of ready-to-use datasets. It handles downloading and preparing the data and constructing a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

Note: Do not confuse [TFDS](https://www.tensorflow.org/datasets) (this library) with [tf.data](https://www.tensorflow.org/guide/data) (TensorFlow API to build efficient data pipelines). TFDS is a high level wrapper around `tf.data`.


Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/datasets/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Installation

`pip install nlp`

Note that `nlp` expects you to have TensorFlow already installed, and currently depends on `tensorflow` (or `tensorflow-gpu`) >= `1.15.0`.

In [0]:
!pip install -q tensorflow nlp matplotlib

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf

import nlp
nlp.disable_progress_bar()

## Citation

Please include the following citation when using `nlp` for a paper, in addition to any citation specific to the used datasets.

```
@misc{TFDS,
  title = { {TensorFlow Datasets}, A collection of ready-to-use datasets},
  howpublished = {\url{https://www.tensorflow.org/datasets}},
}
```

## Eager execution

TensorFlow Datasets is compatible with both TensorFlow [Eager mode](https://www.tensorflow.org/guide/eager) and Graph mode. For this colab, we'll run in Eager mode.

In [0]:
tf.enable_v2_behavior()

## List the available datasets

Each dataset is implemented as a [`nlp.DatasetBuilder`](https://www.tensorflow.org/datasets/api_docs/python/nlp/core/DatasetBuilder) and you can list all available builders with `nlp.list_builders()`.

You can see all the datasets with additional documentation on the [datasets documentation page](https://www.tensorflow.org/datasets/catalog/overview).

In [0]:
nlp.list_builders()

## `nlp.load`: A dataset in one line

[`nlp.load`](https://www.tensorflow.org/datasets/api_docs/python/nlp/load) is a convenience method that's the simplest way to build and load a `tf.data.Dataset`.

`tf.data.Dataset` is the standard TensorFlow API to build input pipelines. If you're not familiar with this API, we **strongly** encourage you to read [the official TensorFlow guide](https://www.tensorflow.org/guide/datasets).

Below, we load the MNIST training data. It downloads and prepares the data, unless you specify `download=False`. Note that once data has been prepared, subsequent calls of `load` will reuse the prepared data.
You can customize where the data is saved/loaded by specifying `data_dir=` (
defaults to `~/nlp/`).

In [0]:
ds_train = nlp.load(name="mnist", split="train")
assert isinstance(ds_train, tf.data.Dataset)
print(ds_train)

When loading a dataset, the canonical default version is used. It is however recommended to specify the major version of the dataset to use, and to advertise which version of the dataset was used in your results. See the
[documentation on datasets versioning](https://github.com/tensorflow/datasets/blob/master/docs/) for more details.

In [0]:
ds_all = nlp.load("mnist:3.*.*")

## Feature dictionaries

All `nlp` datasets contain feature dictionaries mapping feature names to Tensor values. A typical dataset, like MNIST, will have 2 keys: `"image"` and `"label"`. Below we inspect a single example.

Note: In graph mode, see the [tf.data guide](https://www.tensorflow.org/guide/datasets#creating_an_iterator) to understand how to iterate on a `tf.data.Dataset`.

In [0]:
for example in ds_train.take(1):  # Only take a single example
  image, label = example["image"], example["label"]

  plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
  print("Label: %d" % label.numpy())

## `DatasetBuilder`

`nlp.load` is really a thin conveninence wrapper around `DatasetBuilder`. We can accomplish the same as above directly with the MNIST `DatasetBuilder`.

In [0]:
mnist_builder = nlp.builder("mnist")
mnist_builder.download_and_prepare()
ds_train = mnist_builder.as_dataset(split="train")
ds_train

## Input pipelines

Once you have a `tf.data.Dataset` object, it's simple to define the rest of an input pipeline suitable for model training by using the [`tf.data` API](https://www.tensorflow.org/guide/datasets).

Here we'll repeat the dataset so that we have an infinite stream of examples, shuffle, and create batches of 32.

In [0]:
ds_train = ds_train.repeat().shuffle(1024).batch(32)

# prefetch will enable the input pipeline to asynchronously fetch batches while
# your model is training.
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

# Now you could loop over batches of the dataset and train
# for batch in ds_train:
#   ...

## DatasetInfo

After generation, the builder contains useful information on the dataset:

In [0]:
info = mnist_builder.info
print(info)

`DatasetInfo` also contains useful information about the features:

In [0]:
print(info.features)
print(info.features["label"].num_classes)
print(info.features["label"].names)

You can also load the `DatasetInfo` directly with `nlp.load` using `with_info=True`.

In [0]:
ds_test, info = nlp.load("mnist", split="test", with_info=True)
print(info)

## Visualization

For image classification datasets, you can use `nlp.show_examples` to display some examples.

In [0]:
fig = nlp.show_examples(info, ds_test)